# Tool Calling Agent with More LLM Models

- Author: [JoonHo Kim](https://github.com/jhboyo)
- Design: []()
- Peer Review :
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/06-DocumentLoader/04-CSV-Loader.ipynb) [![Open in GitHub](https://img.shields.io/badge/Open%20in%20GitHub-181717?style=flat-square&logo=github&logoColor=white)](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/06-DocumentLoader/04-CSV-Loader.ipynb)


## Overview
It is not limited to `OpenAI` models but also supports implementations from diverse LLM providers such as `Anthropic`, `Google Gemini`, `Together.ai`, `Ollama`, and `Mistral`. This allows developers to leverage the unique characteristics and strengths of each model to create agents optimized for the specific requirements of their applications.

Key Topics
In this chapter, we will delve into the process of creating and executing tool-calling agents using various `LLMs`. The key topics covered include:

- Tool Selection: How agents choose the most suitable tools for specific tasks.
- `LLM` Integration: Integrating `LLMs` from `OpenAI` and other providers into LangChain to enable agent functionality.
- Agent Creation: Creating agents using LangChain's agent classes.
- Agent Execution: Executing agents to perform tasks.

Objectives
By the end of this chapter, you will be able to:

- How to create and execute tool-calling agents using various `LLMs`.
- Create automated workflows that call various tools using LangChain's agent classes.
- Combine multiple `LLMs` to implement agents with optimized performance.

Now, let’s explore how to maximize productivity using LangChain’s flexible agent framework. 🚀

### Table of Contents

- [Overview](#overview)
- [Environment Setup](#environment-setup)
- [List of LLMs Supporting Tool Calling](#list-of-llms-supporting-tool-calling)
- [Using Multiple LLM Integrations with LangChain](#using-multiple-llm-integrations-with-langchain)
- [Define the tool](#define-the-tool)
- [Generate Prompts for Agents](#generate-prompts-for-agents)
- [Generate an AgentExecutor, run it and review the results](#generate-an-agentexecutor-run-it-and-review-the-results)

### References

- [Tool Calling Agent](https://blog.langchain.dev/tool-calling-with-langchain/)
- [LangChain ChatOpenAI](https://python.langchain.com/docs/integrations/chat/openai/)
- [LangChain ChatAnthropic](https://python.langchain.com/docs/integrations/chat/anthropic/)
- [LangChain ChatGoogleGenerativeAI](https://python.langchain.com/docs/integrations/providers/google/)
- [LangChain ChatOllama](https://python.langchain.com/docs/integrations/providers/ollama/)
- [LangChain ChatTogether](https://python.langchain.com/docs/integrations/providers/together/)
----

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- `langchain-opentutorial` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [`langchain-opentutorial`](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.
- `langchain-ollama` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 

In [1]:
%%capture --no-stderr
%pip install -qU langchain-opentutorial
%pip install -qU langchain-ollama==0.1.3
%pip install -qU feedparser

In [ ]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "ANTHROPIC_API_KEY": "",
        "GOOGLE_API_KEY": "",
        "OLLAMA_API_KEY": "",
        "TOGETHER_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "Tool Calling Agent with More LLM Models",
    }
)

In [22]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langchain",
        "langchain_openai",
        "langchain_anthropic",
        "langchain_google_genai",
        "langchain_ollama",
        "langchain_community",
        "langchain_core"
    ],
    verbose=False,
    upgrade=False,
)


## List of LLMs Supporting Tool Calling

To proceed with the hands-on tutorial, the following setup steps are required:

1. Issue API keys for each `LLM` call.
2. Add the issued key to the `.env` file.


**Anthropic**

- [Anthropic API Key Issuance](https://console.anthropic.com/settings/keys)
- Add the issued key to the `.env` file as `ANTHROPIC_API_KEY`.


**Gemini**

- [Gemini API Key Issuance](https://aistudio.google.com/app/apikey?hl=ko)
- Add the issued key to the `.env` file as `GOOGLE_API_KEY`.


**Ollama**
- [List of Ollama Tool Calling Supported Models](https://ollama.com/search?c=tools)
- Instead of issuing an API Key, [Ollama installation](https://ollama.com) is required to proceed with the ollama tutorial.
- We will use [lama3.1 Model](https://ollama.com/library/llama3.1) for This Tutorial for this tutorial.
- After installing Ollama, run the following commands in the terminal to download the models: `ollama pull llama3.1` and `ollama pull qwen2.5`


**Together AI**
- [Together API Key Issuance](https://api.together.ai/)
- Add the issued key to the `.env` file as `TOGETHER_API_KEY`.

## Using Multiple LLM Integrations with LangChain
This walks you through integrating and configuring various `LLMs` using LangChain, allowing you to experiment with different models from providers like `OpenAI`, `Anthropic`, `Google`, and others.

In [23]:
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
import os

# GPT-4o-mini
gpt = ChatOpenAI(model="gpt-4o-mini")

# Claude-3-5-sonnet
claude = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0)

# Gemini-1.5-pro-latest
gemini = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)

# Llama-3.1-70B-Instruct-Turbo
llama = ChatOpenAI(
    base_url="https://api.together.xyz/v1",
    api_key=os.environ["TOGETHER_API_KEY"],
    model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
)

# Llama-3.1
ollama = ChatOllama(model="llama3.1", temperature=0)

# Qwen2.5 7B 
qwen = ChatOllama(
    model="qwen2.5:latest",
)

## Define the tool

Before defining the tool, we will build some functions to collect and fetch news from Website for keyword that user input.

 `_fetch_news` funtion gets news from given url and return the result as a list of dictionaries.
 * `Args:` url (str) is a url for fetching news. k (int) is a number of news to fetch.(default: 3)
 * `Return:` List[Dict[str, str]] is a list of dictionaries that contains news title and link.


 `_collect_news` return a sorted list of news items.
 * `Args:` news_list (List[Dict[str, str]]) is a list of dictionaries that contains news information.
 * `Return:` List[Dict[str, str]] is a list of dictionaries that contains url and contents.

`search_by_keyword` funtion searches news using keyword and return the result as a list of dictionaries.
 * `Args:` keyword (str) is a keyword to search. k (int) is a number of news to fetch.(default: 3)
 * `Return:` List[Dict[str, str]] is a list of dictionaries that contains url and contents.


In [24]:
from typing import List, Dict, Optional
from urllib.parse import quote
import feedparser

class GoogleNews:

    def _fetch_news(self, url: str, k: int = 3) -> List[Dict[str, str]]:
        news_data = feedparser.parse(url)
        return [
            {"title": entry.title, "link": entry.link}
            for entry in news_data.entries[:k]
        ]
    
    def _collect_news(self, news_list: List[Dict[str, str]]) -> List[Dict[str, str]]:
        if not news_list:
            print("There is no news for the keyword.")
            return []

        result = []
        for news in news_list:
            result.append({"url": news["link"], "content": news["title"]})

        return result

    def search_by_keyword(
            self, keyword: Optional[str] = None, k: int = 3
        ) -> List[Dict[str, str]]:
            
            if keyword:
                encoded_keyword = quote(keyword)
                url = f"https://news.google.com/rss/search?q={encoded_keyword}&hl=en&gl=US&ceid=US:en"
            else:
                url = f"https://news.google.com/rss?hl=en&gl=US&ceid=US:en"

            news_list = self._fetch_news(url, k)
            return self._collect_news(news_list)

Fetch News for keyword that user input from Google News Website.

In [ ]:
from langchain.tools import tool
from typing import List, Dict

# Define the tool
@tool
def search_news(query: str) -> List[Dict[str, str]]:
    """Search Google News by input keyword"""
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)

print(f"Tool Name: {search_news.name}")
print(f"Tool Description: {search_news.description}")


Define Tools

In [27]:
tools = [search_news]

## Generate Prompts for Agents
Prompt is a text that describes the task the model will perform.(Input the tool name and role)


- `chat_history` : A variable that stores previous conversation history (can be omitted if multi-turn support is not required).
- `agent_scratchpad` : A variable for temporary storage used by the agent.
- `input` : The user's input.

In [28]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant."
            "Make sure to use the `search_news` tool for searching keyword related news.",        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

Generate an `Agent` based on an `LLM` (Large Language Model)

In [29]:
gpt_agent = create_tool_calling_agent(gpt, tools, prompt)
claude_agent = create_tool_calling_agent(claude, tools, prompt)
gemini_agent = create_tool_calling_agent(gemini, tools, prompt)
llama_agent = create_tool_calling_agent(llama, tools, prompt)
ollama_agent = create_tool_calling_agent(ollama, tools, prompt)
qwen_agent = create_tool_calling_agent(qwen, tools, prompt)

## Generate an AgentExecutor, run it and review the results


In [ ]:
from langchain.agents import AgentExecutor

# execute gpt_agent
agent_executor = AgentExecutor(
    agent=gemini_agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
)

result = agent_executor.invoke({"input": "Search the latest AI Investment news"})

print("Results of Agent Execution:")
print(result["output"])

Run the agent using various `LLMs` (Large Language Models).

The following is a function that generates and runs an Agent using the provided `LLM` and outputs the results.

In [31]:
def execute_agent(llm, tools, input_text, label):
    
    agent = create_tool_calling_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools, verbose=False)
    result = executor.invoke({"input": input_text})
    print(f"Results from [{label}] ")

    if isinstance(result["output"], list) and len(result["output"]) > 0:
        for item in result["output"]:
            if "text" in item:
                print(item["text"])
    elif isinstance(result["output"], str):
        print(result["output"])
    else:
        print(result["output"])

Generate and run agents for each LLM and outputs the results.

In [32]:
query = (
    "Search for news related to AI investment and write the results in the format of an Instagram post."
)

In [ ]:
# gpt
execute_agent(gpt, tools, query, "gpt")

In [ ]:
# claude
execute_agent(claude, tools, query, "claude")

In [ ]:
# gemini
execute_agent(gemini, tools, query, "gemini")

In [ ]:
# llama3.1 70B (Together.ai)
execute_agent(
    llama,
    tools,
    "Search AI related news and write it in Instagram post format",
    "llama3.1 70B",
)

In [ ]:
# llama3.1 8B (ollama)
execute_agent(ollama, tools, query, "llama3.1(Ollama)")

In [ ]:
# qwen2.5 7B (ollama)
execute_agent(qwen, tools, query, "qwen2.5(Ollama)")